In [1]:
print("jpjfpvj!")

jpjfpvj!


In [82]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from langchain.vectorstores import Qdrant
from qdrant_client import QdrantClient, models
from qdrant_client.http.models import PointStruct
from langchain_qdrant import QdrantVectorStore




In [17]:
PINECONE_API_KEY = "80687416-e304-4be0-9c40-a21662b21daa"
PINECONE_API_ENV = "us-east-1-aws"

In [92]:
from qdrant_client import QdrantClient
record=0
url= "https://00b4cc2e-cb61-49d2-af8c-8c68be993a54.europe-west3-0.gcp.cloud.qdrant.io:6333"
api_key = "WKouwQekjSIp2MWe_YGbNnoVUwbGMbxgGemoY6BXrh8kaUs7DrJMRg"
connection = QdrantClient(
    url= url,
    api_key= api_key,
)

# print(qdrant_client.get_collections())
connection.recreate_collection(
    collection_name="odinsCluster",
    vectors_config=models.VectorParams(size=384, distance=models.Distance.COSINE)
)
print("Create collection response : ", connection)
info = connection.get_collection(collection_name="odinsCluster")

print("Collection info :",info)
for get_info in info:
    print(get_info)

C:\Users\hp\AppData\Local\Temp\ipykernel_6464\2519346245.py:11: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  connection.recreate_collection(


Create collection response :  <qdrant_client.qdrant_client.QdrantClient object at 0x000002D73D873D00>
Collection info : status=<CollectionStatus.GREEN: 'green'> optimizer_status=<OptimizersStatusOneOf.OK: 'ok'> vectors_count=None indexed_vectors_count=0 points_count=0 segments_count=2 config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=384, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None, multivector_config=None), shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, ma

In [63]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [64]:
extracted_data = load_pdf("../data")

In [8]:
# extracted_data

In [65]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [66]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 7020


In [13]:
#  text_chunks

In [67]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [68]:
embeddings_model = download_hugging_face_embeddings()

c:\Users\hp\anaconda3\envs\mchatbot\lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [48]:
embeddings_model

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [69]:
query_result = embeddings_model.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [20]:
#  query_result

In [12]:
# #Initializing the Pinecone
# pinecone.init(api_key=PINECONE_API_KEY, environment=PINECONE_API_ENV)

# index_name="med-chatbot"

# import time

# dimentions= 384
# pinecone.create_index(name=index_name,dimension=dimentions,metric="cosine")
# index= pinecone.Index(index_name=index_name)

# #Creating Embeddings for Each of The Text Chunks & storing

# # docsearch=Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

In [80]:
valid_text_chunks = [t.page_content for t in text_chunks ]


In [112]:
docsearch = QdrantVectorStore.from_texts(
    valid_text_chunks, 
    embedding=embeddings_model,
    url=url, 
    api_key=api_key,  
    collection_name="odinsCluster",
    force_recreate=True
)

In [115]:
#If we already have an index we can load it like this

# docsearch = QdrantVectorStore.from_existing_collection(
#     embedding=embeddings_model,
#     collection_name="odinsCluster",
#     url=url, 
#     api_key=api_key,  
# )

query = "What are Allergies"

docs=docsearch.search(query, "similarity", k=3)

print("Result", docs)




Result [Document(metadata={'_id': 'c4c155fc-f8df-4b2c-a39e-087437f1f119', '_collection_name': 'odinsCluster'}, page_content="GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies\nAllergic rhinitis is commonly triggered by\nexposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen.\nThe presence of an allergen causes the\nbody's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE.\nIgE molecules attach to mast\ncells, which contain histamine.HistaminePollen grains\nLymphocyte\nFIRST EXPOSURE"), Document(metadata={'_id': '5d49d2c1-09d2-4635-b327-20927b960205', '_collection_name': 'odinsCluster'}, page_content='allergens are the following:\n• plant pollens\n• animal fur and dander\n• body parts from house mites (microscopic creatures\nfound in all houses)\n• house dust• mold spores• cigarette smoke• solvents• cleaners\nCommon food allergens include the foll

In [116]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [117]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [118]:
llm=CTransformers(model="../model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [120]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

ValidationError: 1 validation error for RetrievalQA
retriever
  Can't instantiate abstract class BaseRetriever with abstract methods _aget_relevant_documents, _get_relevant_documents (type=type_error)

In [125]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])